In [1]:
!pip install qiskit qiskit_aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00


In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
def bv_oracle(qc, inputs, ancilla, s, b='0'):

    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

    if b == '1':
        qc.x(ancilla)

In [4]:
def bernstein_vazirani_circuit(s, b='0'):

    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n
    qc.x(ancilla)
    qc.h(ancilla)

    # Apply H on input qubits (create superposition)
    for q in inputs:
        qc.h(q)

    # Oracle for f(x) = s·x ⊕ b
    bv_oracle(qc, inputs, ancilla, s, b)

    # Apply H on input qubits again
    for q in inputs:
        qc.h(q)

    # Measure input register
    qc.measure(inputs, creg)
    return qc

In [5]:
def run_bv(qc, shots=1024, noise_model=None, basis_gates=None):
    sim = AerSimulator()
    if noise_model is not None:
        tqc = transpile(qc, sim, basis_gates=basis_gates)
        job = sim.run(tqc, shots=shots, noise_model=noise_model)
    else:
        tqc = transpile(qc, sim)
        job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most


if __name__ == '__main__':

    s = '1011'
    b = '0'
    print('Secret string s =', s, '    constant b =', b)

    qc = bernstein_vazirani_circuit(s, b)
    print(qc.draw(fold=-1))

    measured = run_bv(qc)
    if measured == s:
        print('Recovered secret string s')
    else:
        print(' Measured string differs from s (noise or error).')

Secret string s = 1011     constant b = 0
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


In [9]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

p1 = 0.001
p2 = 0.01
error1 = depolarizing_error(p1, 1)
error2 = depolarizing_error(p2, 2)
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error1, ['h', 'x', 'u3', 'u2', 'u1'])
noise_model.add_all_qubit_quantum_error(error2, ['cx'])
basis_gates = noise_model.basis_gates

# Run BV with noise
s = '1011'
b = '0'
qc = bernstein_vazirani_circuit(s, b)
run_bv(qc, shots=2048, noise_model=noise_model, basis_gates=basis_gates)


/usr/local/lib/python3.12/dist-packages/qiskit/compiler/transpiler.py:269: UserWarning: Providing `coupling_map` and/or `basis_gates` along with `backend` is not recommended, as this will invalidate the backend's gate durations and error rates.
  pm = generate_preset_pass_manager(


Counts: {'0101': 13, '0000': 7, '1100': 8, '1001': 7, '0001': 13, '1101': 2000}
Most frequent measured bitstring (input register): 1101


'1101'